In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
!pip install wandb

In [3]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/Redes Neurais/Github/Redes-Neurais/Titanic/train_tratado.csv')

In [6]:
train = df_train.drop(columns = ['Transported'])
y = df_train['Transported']

### Normalizar

transformer = Normalizer().fit(train) 
transformer
train=transformer.transform(train)


#Splitar em treinamento e teste

x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.3)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(4624, 27)
(4624,)
(1982, 27)
(1982,)


####Tunar hiperparametro

In [ ]:
wandb.init(settings=wandb.Settings(start_method="thread"),project="teste")

In [ ]:
model = Sequential() 
#model.add(Dropout(0.2))
model.add(Dense(28, activation='relu', input_dim=train.shape[1]))
model.add(Dense(28, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy']) 

In [22]:
sweep_config = {
  'method': 'grid', 
  'metric': {
      'name': 'val_loss',
      'goal': 'minimize'
  },
 'early_terminate':{'type': 'hyperband','min_iter': 5},
  'parameters': {
      #'batch_size': {'values': [100]}
      'learning_rate':{'values': [0.0001]}
      #,'dropout_1':{'values':[0.1,0.3,0.8]}
      #,'dropout_2':{'values':[0.3,0.4,0.5,0.8,0.95]}
      #,'l1_size':{'values':[16,32,50,64,128]}
      #,'l2_size':{'values':[16,32,50,64,128]}
      #,'fc1_size':{'values':[2,4,8,16,32]}
      #,'fc2_size':{'values':[2,4,8,16,32]}
      #,'fc3_size':{'values':[2,4,8,16,32]}
      #,'fc4_size':{'values':[2,4,8,16,32]}
      ,'fc_layer_size': {'values':[256,512,1024,1500]}
      #,'optimizer': {'values': ['adam','adamax']}

  }
}

In [23]:
def sweep_train(config_defaults=None):
    # Set default values
    config_defaults = {
        "batch_size": 64,
        "learning_rate": 0.01
    }
    # Initialize wandb with a sample project name
    wandb.init()  # this gets over-written in the Sweep

    # Specify the other hyperparameters to the configuration, if any
    '''    wandb.config.epochs = 2
    wandb.config.log_step = 20
    wandb.config.val_log_step = 50
    wandb.config.architecture_name = "MLP"
    wandb.config.dataset_name = "MNIST"'''


    model = Sequential() 
    #model.add(Dropout(0.2))
    model.add(Dense(wandb.config.fc_layer_size, activation='relu', input_dim=train.shape[1]))
    model.add(Dense(wandb.config.fc_layer_size, activation='relu'))
    model.add(Dense(wandb.config.fc_layer_size, activation='relu'))
    model.add(Dense(wandb.config.fc_layer_size, activation='relu'))
    model.add(Dense(1, activation='sigmoid')) 

    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=wandb.config.learning_rate), metrics=['accuracy']) 

    

    model.fit(x_train, y_train, epochs=25, validation_data=(x_test, y_test), callbacks=[WandbCallback()])

In [24]:
sweep_id = wandb.sweep(sweep_config, project="teste_LR")

Create sweep with ID: by5iuclj
Sweep URL: https://wandb.ai/lucasalmada/teste_LR/sweeps/by5iuclj


In [25]:
wandb.agent(sweep_id, function=sweep_train,count=10)

wandb: Agent Starting Run: akwge6v2 with config:
wandb: 	fc_layer_size: 256
wandb: 	learning_rate: 0.0001
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/25
140/145 [===========================>..] - ETA: 0s - loss: 0.5932 - accuracy: 0.7415

wandb: Adding directory to artifact (/content/wandb/run-20221206_010016-akwge6v2/files/model-best)... Done. 0.0s


145/145 [==============================] - 3s 16ms/step - loss: 0.5894 - accuracy: 0.7435 - val_loss: 0.4939 - val_accuracy: 0.8007
Epoch 2/25
141/145 [============================>.] - ETA: 0s - loss: 0.4911 - accuracy: 0.7901

wandb: Adding directory to artifact (/content/wandb/run-20221206_010016-akwge6v2/files/model-best)... Done. 0.0s


...

145/145 [==============================] - 12s 79ms/step - loss: 0.4314 - accuracy: 0.8023 - val_loss: 0.4522 - val_accuracy: 0.8012
Epoch 23/25
145/145 [==============================] - 8s 53ms/step - loss: 0.4302 - accuracy: 0.8006 - val_loss: 0.4751 - val_accuracy: 0.7972
Epoch 24/25
145/145 [==============================] - ETA: 0s - loss: 0.4234 - accuracy: 0.8021

wandb: Adding directory to artifact (/content/wandb/run-20221206_010542-3wbszvfr/files/model-best)... Done. 0.8s


145/145 [==============================] - 9s 62ms/step - loss: 0.4234 - accuracy: 0.8021 - val_loss: 0.4411 - val_accuracy: 0.7977
Epoch 25/25
145/145 [==============================] - 8s 53ms/step - loss: 0.4226 - accuracy: 0.8015 - val_loss: 0.4500 - val_accuracy: 0.7926


accuracy,▁▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████▇████
epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▅▅▅▅▄▅▄▄▄▄▃▃▃▃▃▂▂▂▁▂▂▂▁▁
val_accuracy,▄▆▄▇▆█▇▇▃▅▅▄▅▇▇█▅▆▆▅▁▇▄▅▂
val_loss,█▇█▇▇▆▇▆▇▆▆█▄▆▆▄▅▃▃▃▄▃▇▁▃
accuracy,0.80147
best_epoch,23
best_val_loss,0.44109
epoch,24
loss,0.42261
val_accuracy,0.79263


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
wandb.agent('ig4a3led', function=sweep_train, count=10)

wandb: Agent Starting Run: ydphoee3 with config:
wandb: 	architecture_name: MLP
wandb: 	batch_size: 38
wandb: 	dataset_name: MNIST
wandb: 	epochs: 3
wandb: 	learning_rate: 0.018443613969138465
wandb: 	log_step: 26
wandb: 	val_log_step: 99
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/50
163/166 [============================>.] - ETA: 0s - loss: 0.4056 - accuracy: 0.8014

wandb: Adding directory to artifact (/content/wandb/run-20221130_022204-ydphoee3/files/model-best)... Done. 0.1s


166/166 [==============================] - 3s 16ms/step - loss: 0.4058 - accuracy: 0.8019 - val_loss: 0.4872 - val_accuracy: 0.8064
Epoch 2/50
165/166 [============================>.] - ETA: 0s - loss: 0.4101 - accuracy: 0.8068

wandb: Adding directory to artifact (/content/wandb/run-20221130_022204-ydphoee3/files/model-best)... Done. 0.1s


...

wandb: Adding directory to artifact (/content/wandb/run-20221130_023327-cajla11c/files/model-best)... Done. 0.1s


166/166 [==============================] - 2s 12ms/step - loss: 0.1840 - accuracy: 0.9129 - val_loss: 1.1585 - val_accuracy: 0.7640
Epoch 7/50
158/166 [===========================>..] - ETA: 0s - loss: 0.1651 - accuracy: 0.9159

wandb: Adding directory to artifact (/content/wandb/run-20221130_023327-cajla11c/files/model-best)... Done. 0.1s


166/166 [==============================] - 3s 16ms/step - loss: 0.1686 - accuracy: 0.9152 - val_loss: 1.0798 - val_accuracy: 0.7572
Epoch 8/50
166/166 [==============================] - 1s 7ms/step - loss: 0.1564 - accuracy: 0.9228 - val_loss: 1.0969 - val_accuracy: 0.7579
Epoch 9/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1536 - accuracy: 0.9234 - val_loss: 1.2471 - val_accuracy: 0.7519
Epoch 10/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1493 - accuracy: 0.9232 - val_loss: 1.1550 - val_accuracy: 0.7617
Epoch 11/50
166/166 [==============================] - 1s 7ms/step - loss: 0.1611 - accuracy: 0.9190 - val_loss: 1.2671 - val_accuracy: 0.7436
Epoch 12/50
166/166 [==============================] - 1s 5ms/step - loss: 0.1574 - accuracy: 0.9235 - val_loss: 1.1557 - val_accuracy: 0.7572
Epoch 13/50
166/166 [==============================] - 1s 6ms/step - loss: 0.1482 - accuracy: 0.9264 - val_loss: 1.2145 - val_accuracy: 0.7761
Epoch 14/50


accuracy,▅▅▅▁▁▂▅▅▄▆▇▆▄▄▄▄▅▆▆▅▄▃▆▇▆▅▇█▅▄▅▇▃▅▇██▇▇▅
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▄▄▄▇█▆▄▄▅▄▃▂▅▅▆▆▄▃▃▃▅▇▄▂▃▃▂▂▄▅▄█▇▄▂▁▂▂▃▅
val_accuracy,▆▇▇▆▆▅▅▄▃▅██▆▆▁▅▇▇▅▆▇▅▆▆▆▅▇▅▆▆▄▅▃▅▅▅▄▃▄▄
val_loss,▃▃▃▃▂▁▁▄▄▂▃▆▄▅▇▃▃▃▅▇▃▂▄▄▄▆▆█▆▇▅█▃▄▄▆█▇▅▅
accuracy,0.92316
best_epoch,6
best_val_loss,1.07981
epoch,49
loss,0.16343
val_accuracy,0.75113


In [ ]:
sweep_id

'wqux4o69'